In [45]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

In [89]:
filepath = '../data/input/'
filename = 'default of credit card clients.xls'

#load data set
df = pd.read_excel(filepath+filename, index_col=0,skiprows=[0])

In [46]:
#show first 5 rows
df.head()

,X1,X2,X3,X4,X5,X6,X7,X8,X9,X10,...,X15,X16,X17,X18,X19,X20,X21,X22,X23,Y
1,20000,2,2,1,24,2,2,-1,-1,-2,...,0,0,0,0,689,0,0,0,0,1
2,120000,2,2,2,26,-1,2,0,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
3,90000,2,2,2,34,0,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
4,50000,2,2,1,37,0,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
5,50000,1,2,1,57,-1,0,-1,0,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,0


In [90]:
#compute correlation
corr = df.corr()

#plot correlation
corr.style.background_gradient(cmap='coolwarm', axis=None).set_precision(3)

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
LIMIT_BAL,1,0.0248,-0.219,-0.108,0.145,-0.271,-0.296,-0.286,-0.267,-0.249,-0.235,0.285,0.278,0.283,0.294,0.296,0.29,0.195,0.178,0.21,0.203,0.217,0.22,-0.154
SEX,0.0248,1,0.0142,-0.0314,-0.0909,-0.0576,-0.0708,-0.0661,-0.0602,-0.0551,-0.044,-0.0336,-0.0312,-0.0246,-0.0219,-0.017,-0.0167,-0.000242,-0.00139,-0.0086,-0.00223,-0.00167,-0.00277,-0.04
EDUCATION,-0.219,0.0142,1,-0.143,0.175,0.105,0.122,0.114,0.109,0.0975,0.0823,0.0236,0.0187,0.013,-0.000451,-0.00757,-0.0091,-0.0375,-0.03,-0.0399,-0.0382,-0.0404,-0.0372,0.028
MARRIAGE,-0.108,-0.0314,-0.143,1,-0.414,0.0199,0.0242,0.0327,0.0331,0.0356,0.0343,-0.0235,-0.0216,-0.0249,-0.0233,-0.0254,-0.0212,-0.00598,-0.00809,-0.00354,-0.0127,-0.0012,-0.00664,-0.0243
AGE,0.145,-0.0909,0.175,-0.414,1,-0.0394,-0.0501,-0.053,-0.0497,-0.0538,-0.0488,0.0562,0.0543,0.0537,0.0514,0.0493,0.0476,0.0261,0.0218,0.0292,0.0214,0.0228,0.0195,0.0139
PAY_0,-0.271,-0.0576,0.105,0.0199,-0.0394,1,0.672,0.574,0.539,0.509,0.475,0.187,0.19,0.18,0.179,0.181,0.177,-0.0793,-0.0701,-0.0706,-0.064,-0.0582,-0.0587,0.325
PAY_2,-0.296,-0.0708,0.122,0.0242,-0.0501,0.672,1,0.767,0.662,0.623,0.576,0.235,0.235,0.224,0.222,0.221,0.219,-0.0807,-0.059,-0.0559,-0.0469,-0.0371,-0.0365,0.264
PAY_3,-0.286,-0.0661,0.114,0.0327,-0.053,0.574,0.767,1,0.777,0.687,0.633,0.208,0.237,0.227,0.227,0.225,0.222,0.00129,-0.0668,-0.0533,-0.0461,-0.0359,-0.0359,0.235
PAY_4,-0.267,-0.0602,0.109,0.0331,-0.0497,0.539,0.662,0.777,1,0.82,0.716,0.203,0.226,0.245,0.246,0.243,0.239,-0.00936,-0.00194,-0.0692,-0.0435,-0.0336,-0.0266,0.217
PAY_5,-0.249,-0.0551,0.0975,0.0356,-0.0538,0.509,0.623,0.687,0.82,1,0.817,0.207,0.227,0.243,0.272,0.27,0.263,-0.00609,-0.00319,0.00906,-0.0583,-0.0333,-0.023,0.204


In [92]:
#print correlation
corr.loc['default payment next month']

LIMIT_BAL                    -0.153520
SEX                          -0.039961
EDUCATION                     0.028006
MARRIAGE                     -0.024339
AGE                           0.013890
PAY_0                         0.324794
PAY_2                         0.263551
PAY_3                         0.235253
PAY_4                         0.216614
PAY_5                         0.204149
PAY_6                         0.186866
BILL_AMT1                    -0.019644
BILL_AMT2                    -0.014193
BILL_AMT3                    -0.014076
BILL_AMT4                    -0.010156
BILL_AMT5                    -0.006760
BILL_AMT6                    -0.005372
PAY_AMT1                     -0.072929
PAY_AMT2                     -0.058579
PAY_AMT3                     -0.056250
PAY_AMT4                     -0.056827
PAY_AMT5                     -0.055124
PAY_AMT6                     -0.053183
default payment next month    1.000000
Name: default payment next month, dtype: float64

We want to inspect for possible errors in the dataset. These are the legal values for the different variables:

* LIMIT_BAL: $\geq$ 0
* SEX: 1,2
* EDUCATION: 1,2,3,4,5
* MARRIAGE: 1,2,3
* AGE: $\geq$ 0
* PAY_* 0, 2-6: -1, 1, 2, 3, 4, 5, 6, 7, 8, 9
* BILL_AMT* 1-6: $\geq$ 0
* PAY_AMT* 1-6: $\geq$ 0
* default payment next month: 0, 1


In [94]:
#define values with limited number of legal values
variables= ['SEX','EDUCATION','MARRIAGE','PAY_0','PAY_2','PAY_3', 'PAY_4','PAY_5','PAY_6','default payment next month']
legal_values = [[1,2],[1,2,3,4,5],[1,2,3],[-1,1,2,3,4,5,6,7,8,9],\
                [-1,1,2,3,4,5,6,7,8,9],[-1,1,2,3,4,5,6,7,8,9],\
                [-1,1,2,3,4,5,6,7,8,9],[-1,1,2,3,4,5,6,7,8,9],\
                [-1,1,2,3,4,5,6,7,8,9],[0,1]] 

In [95]:
#print count of values for variables with limited possible values
for variable in variables:
    print('Variable '+variable)
    print('Value|Count')
    print(df[variable].value_counts())
    print('\n')

Variable SEX
Value|Count
2    18112
1    11888
Name: SEX, dtype: int64


Variable EDUCATION
Value|Count
2    14030
1    10585
3     4917
5      280
4      123
6       51
0       14
Name: EDUCATION, dtype: int64


Variable MARRIAGE
Value|Count
2    15964
1    13659
3      323
0       54
Name: MARRIAGE, dtype: int64


Variable PAY_0
Value|Count
 0    14737
-1     5686
 1     3688
-2     2759
 2     2667
 3      322
 4       76
 5       26
 8       19
 6       11
 7        9
Name: PAY_0, dtype: int64


Variable PAY_2
Value|Count
 0    15730
-1     6050
 2     3927
-2     3782
 3      326
 4       99
 1       28
 5       25
 7       20
 6       12
 8        1
Name: PAY_2, dtype: int64


Variable PAY_3
Value|Count
 0    15764
-1     5938
-2     4085
 2     3819
 3      240
 4       76
 7       27
 6       23
 5       21
 1        4
 8        3
Name: PAY_3, dtype: int64


Variable PAY_4
Value|Count
 0    16455
-1     5687
-2     4348
 2     3159
 3      180
 4       69
 7       58
 5       3

In [96]:
print('# of entries before clean up: {}'.format(len(df.index)))

df = df.drop(df[(df.BILL_AMT1 == 0) &
                (df.BILL_AMT2 == 0) &
                (df.BILL_AMT3 == 0) &
                (df.BILL_AMT4 == 0) &
                (df.BILL_AMT5 == 0) &
                (df.BILL_AMT6 == 0) &
                (df.PAY_AMT1 == 0) &
                (df.PAY_AMT2 == 0) &
                (df.PAY_AMT3 == 0) &
                (df.PAY_AMT4 == 0) &
                (df.PAY_AMT5 == 0) &
                (df.PAY_AMT6 == 0)].index)

# of entries before clean up: 30000
